In [30]:
import numpy as np, sys, os
sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), "cmake-build-release"))
import pywrap
import random
import qgrid
import table_pb2
import pandas as pd


In [2]:
arr1 = np.repeat(np.arange(100), 10000).astype(np.uint32)
np.random.shuffle(arr1)
arr2 = np.repeat(np.arange(1000000), 1).astype(np.uint32)
np.random.shuffle(arr2)

In [3]:
%%timeit -n 10
v, inv = pywrap.fast_unique(arr1)
assert np.sum(v[inv] != arr1) == 0

6.88 ms ± 738 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
config = table_pb2.Table()
config.name = "securities_minute"
config.schema.tag_column.append("s")
close_price = table_pb2.ColumnSchema()
close_price.name = "c"
close_price.type = table_pb2.ColumnSchema.FLOAT
config.schema.value_column.append(close_price)
table = pywrap.Table("/tmp/first", config.SerializeToString())

In [35]:
for e in range(3, 1000):
    table.append_data({
        "t": np.arange(100000).astype(np.int64), 
        "s": np.ones(100000).astype(np.uint32) * e, 
        "c": np.arange(100000).astype(np.float32) / 10,
    })
    print(e)

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278


In [63]:
table.print_meta()

schema {
  tag_column: "s"
  value_column {
    name: "c"
    type: FLOAT
  }
}
sub_table_id {
  id: "sub,s=GOOG"
  tag {
    key: "s"
    value: 1
  }
  str_tag: "GOOG"
}
sub_table_id {
  id: "sub,s=FB"
  tag {
    key: "s"
    value: 2
  }
  str_tag: "FB"
}
sub_table_id {
  id: "sub,s=0"
  tag {
    key: "s"
    value: 3
  }
  str_tag: "0"
}
sub_table_id {
  id: "sub,s=1"
  tag {
    key: "s"
    value: 4
  }
  str_tag: "1"
}
sub_table_id {
  id: "sub,s=2"
  tag {
    key: "s"
    value: 5
  }
  str_tag: "2"
}
sub_table_id {
  id: "sub,s=3"
  tag {
    key: "s"
    value: 6
  }
  str_tag: "3"
}
sub_table_id {
  id: "sub,s=4"
  tag {
    key: "s"
    value: 7
  }
  str_tag: "4"
}
sub_table_id {
  id: "sub,s=5"
  tag {
    key: "s"
    value: 8
  }
  str_tag: "5"
}
sub_table_id {
  id: "sub,s=6"
  tag {
    key: "s"
    value: 9
  }
  str_tag: "6"
}
sub_table_id {
  id: "sub,s=7"
  tag {
    key: "s"
    value: 10
  }
  str_tag: "7"
}
sub_table_id {
  id: "sub,s=8"
  tag {
    key: "s"

In [89]:
sel = table_pb2.Selector()
sel.column.extend(["t", "s", "c"])
sel.time_selector.CopyFrom(table_pb2.TimeSelector(start=80000, end=80500, include_end=True))
# sel.sub_table_selector.tag_selector.append(table_pb2.TagSelector(name="s", value=["GOOG", "FB"]))
sel.sub_table_selector.tag_order.append("s")
result = table.query(sel.SerializeToString())
v, inv = pywrap.fast_unique(result["s"])
mapping = dict(zip(v, table.resolve_str_refs(v)))
result["s"] = np.array([mapping[e] for e in v], dtype=np.object)[inv]
df = pd.DataFrame(result, columns = sel.column)
print(len(df))
# qgrid.show_grid(df)


499998


In [95]:
%%timeit -n 10
sel = table_pb2.Selector()
sel.column.extend(["t", "s", "c"])
sel.time_selector.CopyFrom(table_pb2.TimeSelector(start=80000, end=80300, include_end=True))
sel.sub_table_selector.tag_selector.append(table_pb2.TagSelector(name="s", value=[str(e) for e in range(1, 500)]))
sel.sub_table_selector.tag_order.append("s")
result = table.query(sel.SerializeToString())
v, inv = pywrap.fast_unique(result["s"])
mapping = dict(zip(v, table.resolve_str_refs(v)))
result["s"] = np.array([mapping[e] for e in v], dtype=np.object)[inv]
pd.DataFrame(result)

34.5 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
sel.time_selector.

SyntaxError: invalid syntax (<ipython-input-10-bc454e085cbf>, line 1)